# Исследования по кластеризации текстовых данных

## Этап 1 - Подготовка 

In [1]:
import os
from pathlib import Path

DATA_DIR = Path("data")
if not DATA_DIR.exists():
    raise Exception("Data directory '%s' not found" % DATA_DIR.absolute())
    
OUT_DIR = Path("out_clustering")
if not OUT_DIR.exists():
    OUT_DIR.mkdir()    

In [2]:
import numpy as np
import pandas as pd

## Этап 2 - Загрузка данных

In [3]:
complaints_file = os.path.join(DATA_DIR, "complaints.xlsx")

raw_data = pd.read_excel(complaints_file, sheet_name="complaints", index_col=None)

In [4]:
raw_data.head(3)

,ISSUENUM,TOPIC,UNDER_TOPIC,REPORTER,ASSIGNEE,CREATOR,STATUS_ISSUE,CREATED,UPDATED,RESOLUTIONDATE,...,FIO_CLIENT,IIN,TEXT_CONTRACT_NUMBER,PHONE_NUMBER,INF_CLIENT_RESPONDING,INCIDENT_REGION,INCIDENT_RESOLVED,FIO_GUILT_WORK,AUTHOR_REGION,AUTHOR_SUBDIVISIONS
0,105578,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,aoryntaeva,akalabaeva,aoryntaeva,Закрыта,2017-12-12 06:47:45,2017-12-12 12:02:50,2017-12-12 12:02:50,...,АҚЖИГИТОВ БЕЙБІТ КЕНЖЕБАЙҰЛЫ,910429301833,0,7776121291,NaN,0,Не требует создания проблемы,NaN,Алматы,Call-center
1,105582,ЖАЛОБА: на корреспонденцию банка,Cмс по предложениям Xsell,atusipova,akalabaeva,atusipova,Закрыта,2017-12-12 08:34:06,2017-12-12 15:41:16,2017-12-12 12:05:45,...,Тверитинова Анна Егоровна,761123400947,3708168706,87078452136 \t,NaN,0,Не требует создания проблемы,NaN,NaN,Call-center
2,105604,ЖАЛОБА: на обслуживание,NaN,azhalgaeva,zissayev,azhalgaeva,Закрыта,2017-12-12 09:27:28,2017-12-21 09:10:29,2017-12-14 16:05:54,...,Дарипова Айгуль Сабыргалиевна,810420403003,0,7292250739,Не дозвон,"Мангистауская область,Бейнеу, ул. Ерконай, д. 1",Не требует создания проблемы,NaN,Актау,Call-center


In [5]:
print("[i] Размерность сырых данных: ", raw_data.shape)

[i] Размерность сырых данных:  (22840, 21)


## Этап 3 - Отбор признаков

In [6]:
valuable_columns = ["ISSUENUM", "TOPIC", "DESCRIPTION"]
raw_data = raw_data[valuable_columns] # Для исследования ограничимся ID 'ISSUENUM', меткой 'TOPIC' и содержанием 'DESCRIPTION'

print("[i] : Информация о данных")
raw_data.info() # Информация о данных

[i] : Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22840 entries, 0 to 22839
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISSUENUM     22840 non-null  int64 
 1   TOPIC        22805 non-null  object
 2   DESCRIPTION  22839 non-null  object
dtypes: int64(1), object(2)
memory usage: 535.4+ KB


In [7]:
#Переименуем столбцы
raw_data.columns = ['issuenum', 'topic', 'description']

## Этап 4 - Проверка на дубли и удаление строк с отсутствующими данными

In [8]:
duplicated = raw_data.loc[raw_data.duplicated(subset=['issuenum'], keep=False)]
print("[i] Количество дублей идентификаторов: ", duplicated['issuenum'].count())

[i] Количество дублей идентификаторов:  0


In [9]:
raw_data = raw_data.dropna(subset=['topic', 'description']) # Удаляем строки с пустыми значениями в столбцах 'TOPIC' и 'DESCRIPTION'

print("[i] Размерность после удаления пустых", raw_data.shape)

[i] Размерность после удаления пустых (22804, 3)


## Этап 5 - Отбор "нужных" обращений

In [10]:
#Отберем только строки, содержащие жалобы, в которых столбец 'TOPIC' содержит различные сочетания слова "жалоба"
raw_data = raw_data[(raw_data.topic.str.contains("ЖАЛОБА: ") |
                     raw_data.topic.str.contains("Жалоба на ") |
                     raw_data.topic.str.contains("Жалобы на "))] #, na=False

print("[i] Размерность данных с Жалобами", raw_data.shape)

[i] Размерность данных с Жалобами (19415, 3)


## Этап 6 - Очистка меток от "мусора"

In [11]:
import re
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # Убрать из темы словосочетания 'ЖАЛОБА: на ',
    text = re.sub( r'ЖАЛОБА: ', '', text) # 'ЖАЛОБА: ',
    text = re.sub( r'Жалоба на ', '', text) # 'Жалоба на ',
    text = re.sub( r'Жалобы на ', '', text) # 'Жалобы на '
    
    text = str.strip(text.lower()) #Переводим в нижний регистр и убираем пробелы в начале и конце
    return  text

In [12]:
raw_data.topic = raw_data.topic.apply(label_cleaner)# Почистим метки от "мусора"

print("[i] Количество категорий (меток):", len(set(raw_data['topic'])))
print("[i] Множество категорий (меток):")
print(set(raw_data["topic"]))

[i] Количество категорий (меток): 9
[i] Множество категорий (меток):
{'корреспонденцию банка', 'услугу "хранитель"', 'обслуживание в терминалах', 'го/филиал/отделения/микроофисы/тт', 'услугу "защита семьи"', 'услугу "страховку"', 'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями', 'карточные продукты', 'обслуживание'}


In [13]:
raw_data.head(3)

,issuenum,topic,description
0,105578,корреспонденцию банка,Клиент просит чтобы банк не отправлял смс о сп...
1,105582,корреспонденцию банка,ДД.\nПри оформлении кредита клиент предоставил...
2,105604,обслуживание,Дарипова Айгуль Сабыргалиевна жалуется на мене...


## Этап 7 - Препроцессинг

In [14]:
#Пишем свой препроцессор (preprocessor)

from nltk.stem.snowball import SnowballStemmer
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    #Удаление незначимых слов
    stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
           'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    remove = r'\b(' + '|'.join(stw) + r')\b'  
    text = re.sub(remove, ' ', text)
    
    text = re.sub( r'\b\w\b', '', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text
%time
raw_data['description'] = raw_data['description'].apply(text_cleaner)

Wall time: 0 ns


In [15]:
raw_data['description'].head(50)

0     клиент прос чтоб банк   отправля смс  спец пре...
1     дд. при оформлен кредит клиент предостав телеф...
2     дарипов айгул сабыргалиевн жал   менеджер банк...
3     добр день! клиент жалуется, что специалист гру...
4     клиент жал     что  лиц отправля смс сообщен ч...
5                                          остаток сумм
6     добр день! клиент оформ денежн кредит. менедже...
7                                              олрробро
12    при оформлен клиент был сказано, что ест акц п...
13    добр день! клиент прос   беспоко   счет спец. ...
14    дд! клиент   довол тем, чтоб получ справк необ...
15    добр день!!! клиент амирбек абза был возмущ от...
16                       кк.   предостав никак информац
17     digit    дан номер приход смс   им эдуард мур...
18    клиент жал   менеджер так как при оформлен тов...
20    клиент прос больш   присыла смс собщен спредло...
21    клиент прос больш   зван   этот номер   говар ...
22    инцидент произошел   город актоб тд мечт  

In [16]:
raw_data.groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             8664
корреспонденцию банка                                                                    7515
услугу "страховку"                                                                        912
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями     772
карточные продукты                                                                        418
го/филиал/отделения/микроофисы/тт                                                         372
услугу "хранитель"                                                                        367
обслуживание в терминалах                                                                 324
услугу "защита семьи"                                                                      71
Name: issuenum, dtype: int64

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

n_clusters=9

vectorizer = TfidfVectorizer() # preprocessor=text_cleaner
print(vectorizer)
print("\n")
clusterer = KMeans(n_clusters=n_clusters)
#clusterer = KMeans(n_clusters=n_clusters, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
print(clusterer)

TfidfVectorizer()


KMeans(n_clusters=9)


In [18]:
content = raw_data.description.tolist()
len(content)

19415

In [19]:
print("[i] обучение кластеризатора...")

text_clstz = Pipeline([
                ('tfidf', vectorizer),
                ('km', clusterer)
                #( 'km', KMeans(n_clusters=n_clusters, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0) )
                ])
text_clstz.fit(content)

print("[i] обучение закончено!")

raw_data['cluster'] = text_clstz.predict(content)

print("[i] разделение на кластеры завершено!")

[i] обучение кластеризатора...
[i] обучение закончено!
[i] разделение на кластеры завершено!


In [20]:
raw_data[raw_data['cluster'] == 0].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             1524
корреспонденцию банка                                                                     330
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями     146
карточные продукты                                                                         88
го/филиал/отделения/микроофисы/тт                                                          68
обслуживание в терминалах                                                                  53
услугу "страховку"                                                                         25
услугу "хранитель"                                                                         16
Name: issuenum, dtype: int64

In [21]:
raw_data[raw_data['cluster'] == 1].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             2616
корреспонденцию банка                                                                    1179
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями     475
карточные продукты                                                                        232
обслуживание в терминалах                                                                 217
услугу "страховку"                                                                        105
го/филиал/отделения/микроофисы/тт                                                         103
услугу "хранитель"                                                                         70
услугу "защита семьи"                                                                       6
Name: issuenum, dtype: int64

In [22]:
raw_data[raw_data['cluster'] == 2].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
корреспонденцию банка                                                                    1846
обслуживание                                                                              125
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями       4
карточные продукты                                                                          2
го/филиал/отделения/микроофисы/тт                                                           2
услугу "страховку"                                                                          1
Name: issuenum, dtype: int64

In [23]:
raw_data[raw_data['cluster'] == 3].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             1823
услугу "страховку"                                                                        577
услугу "хранитель"                                                                        192
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями      75
услугу "защита семьи"                                                                      49
карточные продукты                                                                         19
корреспонденцию банка                                                                      14
го/филиал/отделения/микроофисы/тт                                                           5
Name: issuenum, dtype: int64

In [24]:
raw_data[raw_data['cluster'] == 4].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
корреспонденцию банка                                                                    1803
карточные продукты                                                                         12
обслуживание                                                                                9
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями       5
услугу "страховку"                                                                          1
Name: issuenum, dtype: int64

In [25]:
raw_data[raw_data['cluster'] == 5].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             181
обслуживание в терминалах                                                                 42
корреспонденцию банка                                                                     39
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями     33
карточные продукты                                                                        32
услугу "страховку"                                                                        28
услугу "хранитель"                                                                        15
го/филиал/отделения/микроофисы/тт                                                          7
услугу "защита семьи"                                                                      6
Name: issuenum, dtype: int64

In [26]:
raw_data[raw_data['cluster'] == 6].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             615
услугу "страховку"                                                                       171
услугу "хранитель"                                                                        70
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями     12
услугу "защита семьи"                                                                     10
карточные продукты                                                                         9
корреспонденцию банка                                                                      2
го/филиал/отделения/микроофисы/тт                                                          1
Name: issuenum, dtype: int64

In [27]:
raw_data[raw_data['cluster'] == 7].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
обслуживание                                                                             1761
го/филиал/отделения/микроофисы/тт                                                         186
карточные продукты                                                                         23
не согласие с условиями договора, %%, задолженностью, штрафами, тарифами и комиссиями      22
корреспонденцию банка                                                                      14
обслуживание в терминалах                                                                  12
услугу "хранитель"                                                                          4
услугу "страховку"                                                                          4
Name: issuenum, dtype: int64

In [28]:
raw_data[raw_data['cluster'] == 8].groupby(['topic'])['issuenum'].count().sort_values(ascending=False)

topic
корреспонденцию банка    2288
обслуживание               10
карточные продукты          1
Name: issuenum, dtype: int64